# IoT Intrusion Detection

The N-BaIoT Dataset contains traffic data for 9 IoT devices. The data comprise of both benign traffic and of a variety of malicious attacks. Here we run three deep neural networks to identify cyberattacks on a Provision PT-737E Security Camera.

In [1]:
import numpy as np
import pandas as pd

In [2]:
benign=pd.read_csv('../input/nbaiot-dataset/5.benign.csv')
g_c=pd.read_csv('../input/nbaiot-dataset/5.gafgyt.combo.csv')
g_j=pd.read_csv('../input/nbaiot-dataset/5.gafgyt.junk.csv')
g_s=pd.read_csv('../input/nbaiot-dataset/5.gafgyt.scan.csv')
g_t=pd.read_csv('../input/nbaiot-dataset/5.gafgyt.tcp.csv')
g_u=pd.read_csv('../input/nbaiot-dataset/5.gafgyt.udp.csv')
m_a=pd.read_csv('../input/nbaiot-dataset/5.mirai.ack.csv')
m_sc=pd.read_csv('../input/nbaiot-dataset/5.mirai.scan.csv')
m_sy=pd.read_csv('../input/nbaiot-dataset/5.mirai.syn.csv')
m_u=pd.read_csv('../input/nbaiot-dataset/5.mirai.udp.csv')
m_u_p=pd.read_csv('../input/nbaiot-dataset/5.mirai.udpplain.csv')

benign=benign.sample(frac=0.25,replace=False)
g_c=g_c.sample(frac=0.25,replace=False)
g_j=g_j.sample(frac=0.5,replace=False)
g_s=g_s.sample(frac=0.5,replace=False)
g_t=g_t.sample(frac=0.15,replace=False)
g_u=g_u.sample(frac=0.15,replace=False)
m_a=m_a.sample(frac=0.25,replace=False)
m_sc=m_sc.sample(frac=0.15,replace=False)
m_sy=m_sy.sample(frac=0.25,replace=False)
m_u=m_u.sample(frac=0.1,replace=False)
m_u_p=m_u_p.sample(frac=0.27,replace=False)

benign['type']='benign'
m_u['type']='mirai_udp'
g_c['type']='gafgyt_combo'
g_j['type']='gafgyt_junk'
g_s['type']='gafgyt_scan'
g_t['type']='gafgyt_tcp'
g_u['type']='gafgyt_udp'
m_a['type']='mirai_ack'
m_sc['type']='mirai_scan'
m_sy['type']='mirai_syn'
m_u_p['type']='mirai_udpplain'

data=pd.concat([benign,m_u,g_c,g_j,g_s,g_t,g_u,m_a,m_sc,m_sy,m_u_p],
               axis=0, sort=False, ignore_index=True)

In [3]:
#how many instances of each class
data.groupby('type')['type'].count()

type
benign            15538
gafgyt_combo      15345
gafgyt_junk       15449
gafgyt_scan       14648
gafgyt_tcp        15676
gafgyt_udp        15602
mirai_ack         15138
mirai_scan        14517
mirai_syn         16436
mirai_udp         15625
mirai_udpplain    15304
Name: type, dtype: int64

In [4]:
#shuffle rows of dataframe 
sampler=np.random.permutation(len(data))
data=data.take(sampler)
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,type
151026,30.648348,73.786284,2.946211,42.920081,72.385358,20.000020,218.499568,69.598291,42.315946,3076.945478,...,0.0,0.0,1.000000,74.0,0.000000e+00,74.0,0.000000e+00,0.0,0.0,mirai_syn
95081,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.0,0.000000e+00,0.0,0.0,gafgyt_udp
62708,45.178681,74.044614,1.078122,54.393687,74.108212,2.720618,97.158271,74.263852,8.336619,1017.833281,...,0.0,0.0,2.898784,74.0,9.536743e-07,74.0,9.094947e-13,0.0,0.0,gafgyt_scan
136845,21.295007,60.058776,0.807607,33.761283,60.127522,1.947972,145.770837,60.161044,2.588216,1890.731832,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.0,0.000000e+00,0.0,0.0,mirai_scan
31968,170.299836,74.006022,0.144488,281.295874,74.014360,0.344454,835.860542,74.032142,1.291707,5257.659324,...,0.0,0.0,1.000000,74.0,0.000000e+00,74.0,0.000000e+00,0.0,0.0,gafgyt_combo


In [5]:
#dummy encode labels, store separately
labels_full=pd.get_dummies(data['type'], prefix='type')
labels_full.head()

,type_benign,type_gafgyt_combo,type_gafgyt_junk,type_gafgyt_scan,type_gafgyt_tcp,type_gafgyt_udp,type_mirai_ack,type_mirai_scan,type_mirai_syn,type_mirai_udp,type_mirai_udpplain
151026,0,0,0,0,0,0,0,0,1,0,0
95081,0,0,0,0,0,1,0,0,0,0,0
62708,0,0,0,1,0,0,0,0,0,0,0
136845,0,0,0,0,0,0,0,1,0,0,0
31968,0,1,0,0,0,0,0,0,0,0,0


In [6]:
#drop labels from training dataset
data=data.drop(columns='type')
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
151026,30.648348,73.786284,2.946211,42.920081,72.385358,20.000020,218.499568,69.598291,42.315946,3076.945478,...,0.000000e+00,0.0,0.0,1.000000,74.0,0.000000e+00,74.0,0.000000e+00,0.0,0.0
95081,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,...,0.000000e+00,0.0,0.0,1.000000,60.0,0.000000e+00,60.0,0.000000e+00,0.0,0.0
62708,45.178681,74.044614,1.078122,54.393687,74.108212,2.720618,97.158271,74.263852,8.336619,1017.833281,...,1.818989e-12,0.0,0.0,2.898784,74.0,9.536743e-07,74.0,9.094947e-13,0.0,0.0
136845,21.295007,60.058776,0.807607,33.761283,60.127522,1.947972,145.770837,60.161044,2.588216,1890.731832,...,0.000000e+00,0.0,0.0,1.000000,60.0,0.000000e+00,60.0,0.000000e+00,0.0,0.0
31968,170.299836,74.006022,0.144488,281.295874,74.014360,0.344454,835.860542,74.032142,1.291707,5257.659324,...,0.000000e+00,0.0,0.0,1.000000,74.0,0.000000e+00,74.0,0.000000e+00,0.0,0.0


In [7]:
#standardize numerical columns
def standardize(df,col):
    df[col]= (df[col]-df[col].mean())/df[col].std()

data_st=data.copy()
for i in (data_st.iloc[:,:-1].columns):
    standardize (data_st,i)

data_st.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
151026,-0.692430,-0.509400,-0.601542,-0.724490,-0.545518,-0.623738,-0.344585,-0.598787,-0.640948,0.171540,...,-0.098909,-0.097904,-0.071396,-0.197790,-0.383413,-0.127628,-0.405197,-0.09854,-0.095264,0.0
95081,-1.150187,-0.615149,-0.601668,-1.136606,-0.645077,-0.624537,-1.118935,-0.679878,-0.642534,-1.137453,...,-0.098909,-0.097904,-0.071396,-0.197790,-0.463774,-0.127628,-0.484507,-0.09854,-0.095264,0.0
62708,-0.468088,-0.507418,-0.601622,-0.611693,-0.531669,-0.624429,-0.776589,-0.559370,-0.642222,-0.704732,...,-0.098909,-0.097904,-0.071396,-0.196798,-0.383413,-0.127628,-0.405197,-0.09854,-0.095264,0.0
136845,-0.836841,-0.614698,-0.601634,-0.814530,-0.644052,-0.624460,-0.603516,-0.678517,-0.642437,-0.333263,...,-0.098909,-0.097904,-0.071396,-0.197790,-0.463774,-0.127628,-0.484507,-0.09854,-0.095264,0.0
31968,1.463724,-0.507714,-0.601662,1.618982,-0.532424,-0.624524,1.853367,-0.561328,-0.642486,1.099560,...,-0.098909,-0.097904,-0.071396,-0.197790,-0.383413,-0.127628,-0.405197,-0.09854,-0.095264,0.0


In [8]:
#training data for the neural net
train_data_st=data_st.values
train_data_st

array([[-0.69243007, -0.50939981, -0.60154247, ..., -0.09854039,
        -0.09526363,  0.        ],
       [-1.15018689, -0.61514895, -0.60166822, ..., -0.09854039,
        -0.09526363,  0.        ],
       [-0.46808843, -0.50741826, -0.6016222 , ..., -0.09854039,
        -0.09526363,  0.        ],
       ...,
       [ 1.48184073,  0.89029963,  1.99298099, ..., -0.09854039,
        -0.09526363,  0.        ],
       [-1.15018689, -0.61514895, -0.60166822, ..., -0.09854039,
        -0.09526363,  0.        ],
       [ 0.1955008 ,  1.45423145,  1.97682988, ..., -0.09854039,
        -0.09526363,  0.        ]])

In [9]:
#labels for training
labels=labels_full.values
labels

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=uint8)

### Keras model

In [10]:
#import libraries
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping


# test/train split  25% test
x_train_st, x_test_st, y_train_st, y_test_st = train_test_split(
    train_data_st, labels, test_size=0.25, random_state=42)

#  create and fit model
model = Sequential()
model.add(Dense(10, input_dim=train_data_st.shape[1], activation='relu'))
model.add(Dense(40, input_dim=train_data_st.shape[1], activation='relu'))
model.add(Dense(10, input_dim=train_data_st.shape[1], activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(labels.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto')
model.fit(x_train_st,y_train_st,validation_data=(x_test_st,y_test_st),
          callbacks=[monitor],verbose=2,epochs=500)

Epoch 1/500
3968/3968 - 5s - loss: 1.2323 - val_loss: 0.9187
Epoch 2/500
3968/3968 - 5s - loss: 0.7908 - val_loss: 0.6809
Epoch 3/500
3968/3968 - 5s - loss: 0.6322 - val_loss: 0.5669
Epoch 4/500
3968/3968 - 5s - loss: 0.5461 - val_loss: 0.5487
Epoch 5/500
3968/3968 - 5s - loss: 0.4919 - val_loss: 0.5169
Epoch 6/500
3968/3968 - 5s - loss: 0.4496 - val_loss: 0.4348
Epoch 7/500
3968/3968 - 5s - loss: 0.4177 - val_loss: 0.4073
Epoch 8/500
3968/3968 - 5s - loss: 0.3915 - val_loss: 0.3897
Epoch 9/500
3968/3968 - 5s - loss: 0.3699 - val_loss: 0.4252
Epoch 10/500
3968/3968 - 5s - loss: 0.3524 - val_loss: 0.3993
Epoch 11/500
3968/3968 - 5s - loss: 0.3503 - val_loss: 0.3462
Epoch 12/500
3968/3968 - 5s - loss: 0.3290 - val_loss: 0.3097
Epoch 13/500
3968/3968 - 5s - loss: 0.3200 - val_loss: 0.3218
Epoch 14/500
3968/3968 - 5s - loss: 0.3090 - val_loss: 0.2964
Epoch 15/500
3968/3968 - 5s - loss: 0.3086 - val_loss: 0.3308
Epoch 16/500
3968/3968 - 5s - loss: 0.2947 - val_loss: 0.3019
Epoch 17/500
3968

In [11]:
# metrics
pred_st = model.predict(x_test_st)
pred_st = np.argmax(pred_st,axis=1)
y_eval_st = np.argmax(y_test_st,axis=1)
score_st = metrics.accuracy_score(y_eval_st, pred_st)
print("accuracy: {}".format(score_st))

accuracy: 0.8896975425330813


In [12]:
#second model
model2 = Sequential()
model2.add(Dense(32, input_dim=train_data_st.shape[1], activation='relu'))
model2.add(Dense(72, input_dim=train_data_st.shape[1], activation='relu'))
model2.add(Dense(32, input_dim=train_data_st.shape[1], activation='relu'))
model2.add(Dense(1, kernel_initializer='normal'))
model2.add(Dense(labels.shape[1],activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto')
model2.fit(x_train_st,y_train_st,validation_data=(x_test_st,y_test_st),
          callbacks=[monitor],verbose=2,epochs=100)

Epoch 1/100
3968/3968 - 6s - loss: 1.1881 - val_loss: 0.8779
Epoch 2/100
3968/3968 - 6s - loss: 0.7484 - val_loss: 0.6438
Epoch 3/100
3968/3968 - 6s - loss: 0.5990 - val_loss: 0.5672
Epoch 4/100
3968/3968 - 5s - loss: 0.5026 - val_loss: 0.4730
Epoch 5/100
3968/3968 - 6s - loss: 0.4486 - val_loss: 0.3994
Epoch 6/100
3968/3968 - 5s - loss: 0.3963 - val_loss: 0.3910
Epoch 7/100
3968/3968 - 6s - loss: 0.3632 - val_loss: 0.3955
Epoch 8/100
3968/3968 - 6s - loss: 0.3304 - val_loss: 0.3233
Epoch 9/100
3968/3968 - 6s - loss: 0.2972 - val_loss: 0.2809
Epoch 10/100
3968/3968 - 6s - loss: 0.2647 - val_loss: 0.2606
Epoch 11/100
3968/3968 - 6s - loss: 0.2429 - val_loss: 0.2323
Epoch 12/100
3968/3968 - 5s - loss: 0.2218 - val_loss: 0.2115
Epoch 13/100
3968/3968 - 6s - loss: 0.2138 - val_loss: 0.2433
Epoch 14/100
3968/3968 - 5s - loss: 0.2023 - val_loss: 0.1998
Epoch 15/100
3968/3968 - 6s - loss: 0.2120 - val_loss: 0.2034
Epoch 16/100
3968/3968 - 6s - loss: 0.1921 - val_loss: 0.2015
Epoch 17/100
3968

In [13]:
# metrics
pred_st = model2.predict(x_test_st)
pred_st = np.argmax(pred_st,axis=1)
y_eval_st = np.argmax(y_test_st,axis=1)
score_st = metrics.accuracy_score(y_eval_st, pred_st)
print("accuracy: {}".format(score_st))

accuracy: 0.902835538752363
